# Parallel multiprocessing data acquisition test

We are going to make a prediction model with market book data also. Which we will gather in real time with the lowest latency possible. To achieve lowest latency we will compare several approaches.

**This process consists with 3 basic tasks:**
1. get data as python dictionary, 
2. convert it to `pandas.DataFrame` 
3. save data to file with pandas method.

Sequential approach `sequential` will be a baseline, where it will be done sequentially.

Other multiprocessing approaches will be a different combinations of what processes do and how do the communicate.

All methods are in `'scripts/'`.  
**Methods to test:**
1. `mp_simple_pipe` - 2 processes connected with `Pipe()`
    * Process 1: Get data, save to df
    * Communication: Pipe()
    * Process 2: save df to csv
2. `mp_simple_queue` - 2 processes connected with `Queue()`
    * Process 1: Get data, save to df
    * Communication: Queue()
    * Process 2: save df to csv
3. `mp_complex_pipe` - Get data process connected to 2 parallel save to df processes with pipes. Latter - with save to csv with another pipe.
    * Process 1: Get data
        * Communication: Pipe()
        * Process 2_0: save to df
        * Communication: Pipe()
        * Process 2_1: save to df
    * Communication: Pipe()
    * Process 3: save df to csv
4. `mp_complex_queue` - Get data process adds data to queue connected to 2 save to df processes. Latter connected with save to df with queue also.
    * Process 1: Get data
        * Communication: Queue()
        * Process 2_0: save to df
        * Process 2_1: save to df
    * Communication: Queue()
    * Process 3: save df to csv
5. `mp_complex_2_queues` - Get data process dump data to 1 queue switching with another in order minimize queue blocking. And then to save to df and save to csv process in one.
    * Process 1: Get data
    * Communication: Queue() x2
    * Process 2: save to df,save df to csv
6. `mp_complex_2_queues_to_1` - Get data process alternates between 2 queues as before, those connected to conveyor process that puts data on another queue which goes to save to df and to csv process.
    * Process 1: Get data
    * Communication: Queue()
    * Communication: Queue()
    * Process 2: conveyor 2 queues to 1
    * Communication: Queue()    
    * Process 2: save to df,save df to csv

**File size**  
Every method will be tested with different memory_threshold as a csv file size. As 1e+06, 1e+07, 1e+08 bytes. Around 1.5, 15, 90 mb.

**Comparison**  
We will measure time between market book data chunks, as 1 data acquisition equals 1 chunk (40 rows). And compare them on the base of mean, std, different quantiles, min and max looking for stability (how far outliers are, q99, std and max) and lowest mean latency (mean,median).

**Data generation for analyze**  
Here we will analyze already generated data by `'scripts/mp_latency_test.py'`. Test time for how long those methods will gather data is specified in that python code file. We should expect different amount of chunks (rows and files) saved as methods processes are terminated abruptly in `test_time` seconds.

## Import

In [1]:
import datetime as dt
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import MetaTrader5 as mt5


from os import listdir
from os.path import isfile, join


# from pandas.plotting import register_matplotlib_converters
# register_matplotlib_converters()

print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.37


**Ticker**

https://www.moex.com/ru/derivatives/equity/indices/  
Site with actual RTS futures ticker.

In [2]:
# ticker = 'RIU2'

In [3]:
%%html
<style>
table {float:left}
</style>

In [4]:
# # terminal init
# if not mt5.initialize():
#     print("initialize() failed, error code =",mt5.last_error())
#     quit()
# else:
#     print(mt5.last_error())

## Functions

In [5]:

def generate_paths():
    '''
    Creates dictionary with name of the method as key and path to csv files as value.
    '''
    memory_thresholds = [1e+6,1e+7,1e+8]
    names = ['sequential',
             'mp_simple_pipe',
             'mp_simple_queue',
             'mp_complex_pipe',
             'mp_complex_queue',
             'mp_complex_2_queues',
             'mp_complex_2_queues_to_1']
    result = {}
    for name in names:
        for memory_threshold in memory_thresholds:
            result[f'{name}_{memory_threshold:.1e}'] = join('data',name,f'{memory_threshold:.1e}')
    return result
# generate_paths()

In [6]:
def file_number(str_):
    '''
    Key for ordering csv files numericaly, by file name.
    '''
    return int(str_.split('\\')[-1][:-4])

In [7]:
# FUNCTION TO GET PATHS TO CSVs

def get_paths_to_csvs(path_to_csvs,sort=True):
    '''
    Returns list with paths to csv files and orders them numericaly if sort=True
    '''
    result = [join(path_to_csvs, f) for f in listdir(path_to_csvs) if isfile(join(path_to_csvs, f))]
    if sort:
        result.sort(key=file_number) # function above
    return result

# test_path = 'data/sequential/'
# get_paths_to_csvs(test_path)

In [8]:
# FUNCTION TO GET INFO ABOUT FILES COUNT, IS TIME ORDERED OR NOT AND ROW COUNT
def df_info(all_files,name):
    '''
    Receives a list with csvs and a name of a method.
    
    Returns df with method name, number of files, is time order kept by the method or not (as 0 and 1) and row count.
    '''
    df_list = []

    for filename in all_files:
        df = pd.read_csv(filename)
        df_list.append(df)

    df = pd.concat(df_list, axis=0, ignore_index=True)
    df['time_after'] = pd.to_datetime(df['time_after'])
    df['time_before'] = pd.to_datetime(df['time_before'])
    df['delta'] = df['time_after'] - df['time_before']
    df['time_mean'] = df['time_before'] + (df['delta']/2)
    
    min_,max_ = df['time_mean'].min(),df['time_mean'].max()
    
    result = pd.DataFrame({'files_n':len(all_files),
                           'time_order':1,
                           'row_count':df.shape[0],
                           'time_delta': max_ - min_
                          },
                         index=[name])    
    
    df_index_before = df.index.copy()
    df = df.sort_values('time_mean',kind='stable')
    
    if df_index_before.equals(df.index):
        # print('Time order is kept')
        pass
    else:
        # print('Time order is invalid')
        result['time_order'] = 0
    
    return result

In [25]:
# FUNCTION TO CALCULATE LATENCY DISTRIBUTION
def concate_files(all_files):
    '''
    Receives a list with csvs.
    
    Returns concatenated df with more columns with time, especially diff - difference between previous and next time.
    '''
    df_list = []

    for filename in all_files:
        df = pd.read_csv(filename)
        df_list.append(df)

    df = pd.concat(df_list, axis=0, ignore_index=True)
    df['time_after'] = pd.to_datetime(df['time_after'])
    df['time_before'] = pd.to_datetime(df['time_before'])
    df['delta'] = df['time_after'] - df['time_before']
    df['time_mean'] = df['time_before'] + (df['delta']/2)
    
    # df_index_before = df.index.copy()
    df = df.sort_values('time_mean',kind='stable')
    
    # if df_index_before.equals(df.index):
    #     # print('Time order is kept')
    # else:
    #     # print('Time order is invalid')
    
    df['diff'] = df['time_mean'].diff()
    
    return df

def calculate_diff_components(df,delete_zero_diffs=True):
    '''
    Receives df with diff column
    Returns another df with diff components (this is what we need to compare).
    
    delete_zero_diffs if preferable, as there are a lot of 0 diffs, because every chunk comes in same time,
    that's about 40 rows with diff = 0. 
    '''
    dt_components = df['diff'].dt.components
    
    if delete_zero_diffs:
        dt_components['bool_sum'] = (dt_components == 0).sum(axis=1)
        dt_components = dt_components[dt_components['bool_sum'] != 7]
        dt_components = dt_components.drop(columns='bool_sum')
    
    dt_components = dt_components.sample(30000)
    
        
    return dt_components

def concate_and_diffs(all_files,delete_zero_diffs=True):
    '''
    Receives list with csv files,
    Returns a combination of previous 2 functions
    '''
    df = concate_files(all_files)
    result = calculate_diff_components(df,delete_zero_diffs)
    return result


In [10]:
def methods_info():
    '''
    Generate methods' info with files_n, time_order is kept or not and row count using df_info().
    '''
    methods_info = pd.DataFrame()
    for name,path in generate_paths().items():
        try:
            result = df_info(get_paths_to_csvs(path),name)
        except Exception as e:
            result = pd.DataFrame({'files_n':0,
                                   'time_order': np.nan},index=[name])
            # print(e)
        methods_info = pd.concat([methods_info,result])
    return methods_info

In [11]:
def save_diff_files_to_csv(path_):
    '''
    Saves diffs dfs to data/diff folder with name of the method. 
    '''
    for name,path in generate_paths().items():
        try:
            concate_and_diffs(get_paths_to_csvs(path)).to_csv(f'{path_}/{name}.csv')
            print(name)
        except:
            continue


In [12]:
def create_dict_with_methods_diff(path_):
    '''
    Creates a dict with method name as a key and diff df as a value.
    '''
    method_diff_time_components_dict = {}
    for path in get_paths_to_csvs(path_,sort=False):
        name = path.split('\\')[-1][:-4]
        method_diff_time_components_dict[name] = pd.read_csv(path)

    return method_diff_time_components_dict    

In [13]:
def create_list_with_diff(method_diff_time_components_dict,return_df=False):
    '''
    Receives dictionary with method name as key and df with diffs as value
    Returns list with df diffs
    Or one concated df if return_df == True
    '''
    df_list = []
    for name, df in method_diff_time_components_dict.items():
        df['name'] = name
        df_list.append(df)
    if return_df:
        return pd.concat(df_list)
    else:
        return df_list


In [14]:
def diff_describe(df_list):
    '''
    Receives list with diffs dfs.
    Returns df with statistics for each method.
    '''
    df_diff_dist = pd.DataFrame()
    for df in df_list:
        # plt.figure(figsize=(12,8))
        # sns.histplot(data=df,x='milliseconds')
        # plt.plot()
        temp_ = pd.DataFrame()
        for col in df.columns.drop(['Unnamed: 0','name']):
            desc = df[col].describe([.25,.5,.75,.99]).reset_index()
            desc['name'] = col
            # desc.index = [str(x)+'_'+col for x in desc.index]
            desc.index = pd.MultiIndex.from_frame(desc[['name','index']])
            desc = desc.drop(columns=['index','name'])
            # display(desc)
            desc.columns = [df['name'][0]]

            temp_ = pd.concat([temp_,desc],axis=0)
        # display(temp_)
        # break
        df_diff_dist = pd.concat([df_diff_dist,temp_.T])
        # break
    return df_diff_dist



In [15]:
def all_together(path_,fresh_data=False):
    methods_info_df = methods_info()

    if fresh_data:
        save_diff_files_to_csv(path_)

    df_dict = create_dict_with_methods_diff(path_)
    df_list = create_list_with_diff(df_dict)
    df_diff_dist = diff_describe(df_list)
    return methods_info_df, df_diff_dist

## Analysis

In [16]:
methods_info_df, df_diff_dist = all_together('data\\diffs\\')
a_list = []
for name,data in df_diff_dist.drop(columns=['days','hours','microseconds','nanoseconds'],level=0).groupby(level=0,axis=1,sort=False):
    a_list.append(data)

In [17]:
methods_info_df

,files_n,time_order,row_count,time_delta
sequential_1.0e+06,412,1.0,7365312.0,0 days 00:09:54.085247
sequential_1.0e+07,27,1.0,4779904.0,0 days 00:09:59.990272500
sequential_1.0e+08,1,1.0,1251392.0,0 days 00:09:59.983577500
mp_simple_pipe_1.0e+06,464,1.0,8300032.0,0 days 00:09:58.792146
mp_simple_pipe_1.0e+07,27,1.0,4821984.0,0 days 00:09:45.614413500
mp_simple_pipe_1.0e+08,0,NaN,NaN,NaT
mp_simple_queue_1.0e+06,461,1.0,8246368.0,0 days 00:09:58.685000
mp_simple_queue_1.0e+07,27,1.0,4821984.0,0 days 00:09:46.823001
mp_simple_queue_1.0e+08,0,NaN,NaN,NaT
mp_complex_pipe_1.0e+06,604,0.0,10804352.0,0 days 00:09:58.260001


* No all methods keep time order and that's expected.
* Not enough test time to make a 100 mb csv (threshold `1e+08`)
* Why is row count so different between different thresholds in 1 method?
* Time delta (that is calculated as `datetime.max - datetime.min`) looks strange in complex queue and lower. Why?
* `mp_complex_2_queues` doesn't work

In [18]:
a_list[0]

name                               minutes                                     \
index                                count      mean       std  min  25%  50%   
mp_complex_2_queues_to_1_1.0e+06   81613.0  0.000012  0.003500  0.0  0.0  0.0   
mp_complex_2_queues_to_1_1.0e+07   61390.0  0.000016  0.004036  0.0  0.0  0.0   
mp_complex_pipe_1.0e+06           337635.0  0.000000  0.000000  0.0  0.0  0.0   
mp_complex_pipe_1.0e+07           167429.0  0.000000  0.000000  0.0  0.0  0.0   
mp_complex_queue_1.0e+06           81613.0  0.000000  0.000000  0.0  0.0  0.0   
mp_complex_queue_1.0e+07           66971.0  0.000000  0.000000  0.0  0.0  0.0   
mp_simple_pipe_1.0e+06            259375.0  0.000000  0.000000  0.0  0.0  0.0   
mp_simple_pipe_1.0e+07            150686.0  0.000000  0.000000  0.0  0.0  0.0   
mp_simple_queue_1.0e+06           257698.0  0.000000  0.000000  0.0  0.0  0.0   
mp_simple_queue_1.0e+07           150686.0  0.000000  0.000000  0.0  0.0  0.0   
sequential_1.0e+06                230165.0  0.000000  0.000000  0.0  0.0  0.0   
sequential_1.0e+07                149371.0  0.000000  0.000000  0.0  0.0  0.0   
sequential_1.0e+08                 39105.0  0.000000  0.000000  0.0  0.0  0.0   

name                                             
index                             75%  99%  max  
mp_complex_2_queues_to_1_1.0e+06  0.0  0.0  1.0  
mp_complex_2_queues_to_1_1.0e+07  0.0  0.0  1.0  
mp_complex_pipe_1.0e+06           0.0  0.0  0.0  
mp_complex_pipe_1.0e+07           0.0  0.0  0.0  
mp_complex_queue_1.0e+06          0.0  0.0  0.0  
mp_complex_queue_1.0e+07          0.0  0.0  0.0  
mp_simple_pipe_1.0e+06            0.0  0.0  0.0  
mp_simple_pipe_1.0e+07            0.0  0.0  0.0  
mp_simple_queue_1.0e+06           0.0  0.0  0.0  
mp_simple_queue_1.0e+07           0.0  0.0  0.0  
sequential_1.0e+06                0.0  0.0  0.0  
sequential_1.0e+07                0.0  0.0  0.0  
sequential_1.0e+08                0.0  0.0  0.0

* `2_queues_to_1` has 1 min delay. Why?

In [19]:
a_list[1]

name                               seconds                                     \
index                                count      mean       std  min  25%  50%   
mp_complex_2_queues_to_1_1.0e+06   81613.0  0.000037  0.010501  0.0  0.0  0.0   
mp_complex_2_queues_to_1_1.0e+07   61390.0  0.000163  0.040360  0.0  0.0  0.0   
mp_complex_pipe_1.0e+06           337635.0  0.000000  0.000000  0.0  0.0  0.0   
mp_complex_pipe_1.0e+07           167429.0  0.000000  0.000000  0.0  0.0  0.0   
mp_complex_queue_1.0e+06           81613.0  0.000000  0.000000  0.0  0.0  0.0   
mp_complex_queue_1.0e+07           66971.0  0.000000  0.000000  0.0  0.0  0.0   
mp_simple_pipe_1.0e+06            259375.0  0.000000  0.000000  0.0  0.0  0.0   
mp_simple_pipe_1.0e+07            150686.0  0.000000  0.000000  0.0  0.0  0.0   
mp_simple_queue_1.0e+06           257698.0  0.000000  0.000000  0.0  0.0  0.0   
mp_simple_queue_1.0e+07           150686.0  0.000000  0.000000  0.0  0.0  0.0   
sequential_1.0e+06                230165.0  0.000000  0.000000  0.0  0.0  0.0   
sequential_1.0e+07                149371.0  0.000174  0.013192  0.0  0.0  0.0   
sequential_1.0e+08                 39105.0  0.000000  0.000000  0.0  0.0  0.0   

name                                              
index                             75%  99%   max  
mp_complex_2_queues_to_1_1.0e+06  0.0  0.0   3.0  
mp_complex_2_queues_to_1_1.0e+07  0.0  0.0  10.0  
mp_complex_pipe_1.0e+06           0.0  0.0   0.0  
mp_complex_pipe_1.0e+07           0.0  0.0   0.0  
mp_complex_queue_1.0e+06          0.0  0.0   0.0  
mp_complex_queue_1.0e+07          0.0  0.0   0.0  
mp_simple_pipe_1.0e+06            0.0  0.0   0.0  
mp_simple_pipe_1.0e+07            0.0  0.0   0.0  
mp_simple_queue_1.0e+06           0.0  0.0   0.0  
mp_simple_queue_1.0e+07           0.0  0.0   0.0  
sequential_1.0e+06                0.0  0.0   0.0  
sequential_1.0e+07                0.0  0.0   1.0  
sequential_1.0e+08                0.0  0.0   0.0

* Are those delays on network side? As sequential method should not have this much.

In [20]:
a_list[2]

name                             milliseconds                                 \
index                                   count       mean       std  min  25%   
mp_complex_2_queues_to_1_1.0e+06      81613.0   0.876968  3.041656  0.0  1.0   
mp_complex_2_queues_to_1_1.0e+07      61390.0   0.793158  4.153593  0.0  0.0   
mp_complex_pipe_1.0e+06              337635.0   1.342088  0.637725  0.0  1.0   
mp_complex_pipe_1.0e+07              167429.0   3.069570  1.766654  0.0  2.0   
mp_complex_queue_1.0e+06              81613.0   0.834071  1.615541  0.0  1.0   
mp_complex_queue_1.0e+07              66971.0   0.846322  1.447216  0.0  1.0   
mp_simple_pipe_1.0e+06               259375.0   1.905330  0.464051  1.0  2.0   
mp_simple_pipe_1.0e+07               150686.0   3.473183  1.679443  1.0  2.0   
mp_simple_queue_1.0e+06              257698.0   1.899274  0.463997  1.0  2.0   
mp_simple_queue_1.0e+07              150686.0   3.494591  1.588832  1.0  2.0   
sequential_1.0e+06                   230165.0   2.182074  5.804234  1.0  2.0   
sequential_1.0e+07                   149371.0   3.438706  4.559226  1.0  2.0   
sequential_1.0e+08                    39105.0  14.946708  7.628490  1.0  9.0   

name                                                       
index                              50%   75%   99%    max  
mp_complex_2_queues_to_1_1.0e+06   1.0   1.0   2.0  547.0  
mp_complex_2_queues_to_1_1.0e+07   1.0   1.0   2.0  921.0  
mp_complex_pipe_1.0e+06            1.0   2.0   2.0  122.0  
mp_complex_pipe_1.0e+07            3.0   5.0   6.0   78.0  
mp_complex_queue_1.0e+06           1.0   1.0   1.0  180.0  
mp_complex_queue_1.0e+07           1.0   1.0   2.0  146.0  
mp_simple_pipe_1.0e+06             2.0   2.0   3.0   16.0  
mp_simple_pipe_1.0e+07             3.0   5.0   7.0   51.0  
mp_simple_queue_1.0e+06            2.0   2.0   3.0   13.0  
mp_simple_queue_1.0e+07            3.0   5.0   6.0   40.0  
sequential_1.0e+06                 2.0   2.0   3.0  178.0  
sequential_1.0e+07                 3.0   5.0   6.0  388.0  
sequential_1.0e+08                15.0  22.0  28.0   37.0

* `mp_simple_queue_1.0e+06` seems to be the most stable with the lowest max and std

In [21]:
temp_ = a_list[2].copy()

for col in temp_.columns:
    temp_[col] = temp_[col] / temp_[col].min()

temp_ = temp_.replace([np.inf, -np.inf], np.nan)    
temp_['sum'] = temp_.iloc[:,1:].sum(axis=1)
temp_

name                             milliseconds                                \
index                                   count       mean        std min 25%   
mp_complex_2_queues_to_1_1.0e+06     2.087022   1.105666   6.555335 NaN NaN   
mp_complex_2_queues_to_1_1.0e+07     1.569876   1.000000   8.951764 NaN NaN   
mp_complex_pipe_1.0e+06              8.634062   1.692081   1.374416 NaN NaN   
mp_complex_pipe_1.0e+07              4.281524   3.870059   3.807467 NaN NaN   
mp_complex_queue_1.0e+06             2.087022   1.051581   3.481790 NaN NaN   
mp_complex_queue_1.0e+07             1.712594   1.067027   3.119020 NaN NaN   
mp_simple_pipe_1.0e+06               6.632784   2.402206   1.000115 NaN NaN   
mp_simple_pipe_1.0e+07               3.853369   4.378926   3.619511 NaN NaN   
mp_simple_queue_1.0e+06              6.589899   2.394570   1.000000 NaN NaN   
mp_simple_queue_1.0e+07              3.853369   4.405918   3.424227 NaN NaN   
sequential_1.0e+06                   5.885820   2.751119  12.509202 NaN NaN   
sequential_1.0e+07                   3.819742   4.335459   9.825979 NaN NaN   
sequential_1.0e+08                   1.000000  18.844541  16.440814 NaN NaN   

name                                                                  sum  
index                              50%   75%   99%        max              
mp_complex_2_queues_to_1_1.0e+06   1.0   1.0   2.0  42.076923   53.737924  
mp_complex_2_queues_to_1_1.0e+07   1.0   1.0   2.0  70.846154   84.797918  
mp_complex_pipe_1.0e+06            1.0   2.0   2.0   9.384615   17.451112  
mp_complex_pipe_1.0e+07            3.0   5.0   6.0   6.000000   27.677525  
mp_complex_queue_1.0e+06           1.0   1.0   1.0  13.846154   21.379525  
mp_complex_queue_1.0e+07           1.0   1.0   2.0  11.230769   19.416817  
mp_simple_pipe_1.0e+06             2.0   2.0   3.0   1.230769   11.633091  
mp_simple_pipe_1.0e+07             3.0   5.0   7.0   3.923077   26.921514  
mp_simple_queue_1.0e+06            2.0   2.0   3.0   1.000000   11.394570  
mp_simple_queue_1.0e+07            3.0   5.0   6.0   3.076923   24.907069  
sequential_1.0e+06                 2.0   2.0   3.0  13.692308   35.952629  
sequential_1.0e+07                 3.0   5.0   6.0  29.846154   58.007592  
sequential_1.0e+08                15.0  22.0  28.0   2.846154  103.131509

## Conclusion 

`mp_simple_queue` with `1.0e+06` file size threshold value is the most stable with the lowest max and std. We will use it to gather market book data.

In [26]:
methods_info_df, df_diff_dist = all_together('data\\diffs2\\',True)
a_list2 = []
for name,data in df_diff_dist.drop(columns=['days','hours','microseconds','nanoseconds'],level=0).groupby(level=0,axis=1,sort=False):
    a_list2.append(data)

sequential_1.0e+06
sequential_1.0e+07
sequential_1.0e+08
mp_simple_pipe_1.0e+06
mp_simple_pipe_1.0e+07
mp_simple_queue_1.0e+06
mp_simple_queue_1.0e+07
mp_complex_pipe_1.0e+06
mp_complex_pipe_1.0e+07
mp_complex_queue_1.0e+06
mp_complex_queue_1.0e+07
mp_complex_2_queues_to_1_1.0e+06
mp_complex_2_queues_to_1_1.0e+07


In [27]:
a_list[2]

name                             milliseconds                                 \
index                                   count       mean       std  min  25%   
mp_complex_2_queues_to_1_1.0e+06      81613.0   0.876968  3.041656  0.0  1.0   
mp_complex_2_queues_to_1_1.0e+07      61390.0   0.793158  4.153593  0.0  0.0   
mp_complex_pipe_1.0e+06              337635.0   1.342088  0.637725  0.0  1.0   
mp_complex_pipe_1.0e+07              167429.0   3.069570  1.766654  0.0  2.0   
mp_complex_queue_1.0e+06              81613.0   0.834071  1.615541  0.0  1.0   
mp_complex_queue_1.0e+07              66971.0   0.846322  1.447216  0.0  1.0   
mp_simple_pipe_1.0e+06               259375.0   1.905330  0.464051  1.0  2.0   
mp_simple_pipe_1.0e+07               150686.0   3.473183  1.679443  1.0  2.0   
mp_simple_queue_1.0e+06              257698.0   1.899274  0.463997  1.0  2.0   
mp_simple_queue_1.0e+07              150686.0   3.494591  1.588832  1.0  2.0   
sequential_1.0e+06                   230165.0   2.182074  5.804234  1.0  2.0   
sequential_1.0e+07                   149371.0   3.438706  4.559226  1.0  2.0   
sequential_1.0e+08                    39105.0  14.946708  7.628490  1.0  9.0   

name                                                       
index                              50%   75%   99%    max  
mp_complex_2_queues_to_1_1.0e+06   1.0   1.0   2.0  547.0  
mp_complex_2_queues_to_1_1.0e+07   1.0   1.0   2.0  921.0  
mp_complex_pipe_1.0e+06            1.0   2.0   2.0  122.0  
mp_complex_pipe_1.0e+07            3.0   5.0   6.0   78.0  
mp_complex_queue_1.0e+06           1.0   1.0   1.0  180.0  
mp_complex_queue_1.0e+07           1.0   1.0   2.0  146.0  
mp_simple_pipe_1.0e+06             2.0   2.0   3.0   16.0  
mp_simple_pipe_1.0e+07             3.0   5.0   7.0   51.0  
mp_simple_queue_1.0e+06            2.0   2.0   3.0   13.0  
mp_simple_queue_1.0e+07            3.0   5.0   6.0   40.0  
sequential_1.0e+06                 2.0   2.0   3.0  178.0  
sequential_1.0e+07                 3.0   5.0   6.0  388.0  
sequential_1.0e+08                15.0  22.0  28.0   37.0

In [23]:
a_list2[2]

name                             milliseconds                                 \
index                                   count       mean       std  min  25%   
mp_complex_2_queues_to_1_1.0e+06        963.0   0.834891  0.398474  0.0  1.0   
mp_complex_2_queues_to_1_1.0e+07        973.0   0.739979  0.585503  0.0  0.0   
mp_complex_pipe_1.0e+06                 962.0   1.349272  0.750057  0.0  1.0   
mp_complex_pipe_1.0e+07                 996.0   3.128514  2.266991  1.0  2.0   
mp_complex_queue_1.0e+06                955.0   0.803141  0.400459  0.0  1.0   
mp_complex_queue_1.0e+07                959.0   0.875912  1.829426  0.0  1.0   
mp_simple_pipe_1.0e+06                  939.0   1.902023  0.434378  1.0  2.0   
mp_simple_pipe_1.0e+07                  997.0   3.469408  1.608311  1.0  2.0   
mp_simple_queue_1.0e+06                 976.0   1.897541  0.422148  1.0  2.0   
mp_simple_queue_1.0e+07                 962.0   3.556133  1.542148  1.0  2.0   
sequential_1.0e+06                      934.0   2.081370  4.603440  1.0  2.0   
sequential_1.0e+07                      932.0   3.357296  1.501886  1.0  2.0   
sequential_1.0e+08                      975.0  14.927179  7.716740  1.0  9.0   

name                                                        
index                              50%   75%    99%    max  
mp_complex_2_queues_to_1_1.0e+06   1.0   1.0   1.38    2.0  
mp_complex_2_queues_to_1_1.0e+07   1.0   1.0   2.00   11.0  
mp_complex_pipe_1.0e+06            1.0   2.0   2.39   10.0  
mp_complex_pipe_1.0e+07            3.0   5.0   6.00   54.0  
mp_complex_queue_1.0e+06           1.0   1.0   1.00    2.0  
mp_complex_queue_1.0e+07           1.0   1.0   2.00   56.0  
mp_simple_pipe_1.0e+06             2.0   2.0   3.00    9.0  
mp_simple_pipe_1.0e+07             3.0   5.0   7.00   11.0  
mp_simple_queue_1.0e+06            2.0   2.0   3.00    8.0  
mp_simple_queue_1.0e+07            3.0   5.0   6.00    7.0  
sequential_1.0e+06                 2.0   2.0   3.00  142.0  
sequential_1.0e+07                 3.0   5.0   6.00   14.0  
sequential_1.0e+08                14.0  22.0  28.00   29.0

In [28]:
a_list2[2]

name                             milliseconds                                 \
index                                   count       mean       std  min  25%   
mp_complex_2_queues_to_1_1.0e+06      29999.0   0.879096  3.759480  0.0  1.0   
mp_complex_2_queues_to_1_1.0e+07      30000.0   0.820767  5.896994  0.0  0.0   
mp_complex_pipe_1.0e+06               30000.0   1.343000  0.592027  0.0  1.0   
mp_complex_pipe_1.0e+07               29999.0   3.076403  1.784538  0.0  2.0   
mp_complex_queue_1.0e+06              29999.0   0.829928  1.518562  0.0  1.0   
mp_complex_queue_1.0e+07              29999.0   0.846862  1.429552  0.0  1.0   
mp_simple_pipe_1.0e+06                30000.0   1.909000  0.494092  1.0  2.0   
mp_simple_pipe_1.0e+07                30000.0   3.464233  1.615841  1.0  2.0   
mp_simple_queue_1.0e+06               30000.0   1.898933  0.459267  1.0  2.0   
mp_simple_queue_1.0e+07               30000.0   3.492600  1.550191  1.0  2.0   
sequential_1.0e+06                    30000.0   2.202333  6.018715  1.0  2.0   
sequential_1.0e+07                    30000.0   3.441400  4.380016  1.0  2.0   
sequential_1.0e+08                    30000.0  14.947800  7.628413  1.0  9.0   

name                                                       
index                              50%   75%   99%    max  
mp_complex_2_queues_to_1_1.0e+06   1.0   1.0   2.0  547.0  
mp_complex_2_queues_to_1_1.0e+07   1.0   1.0   2.0  921.0  
mp_complex_pipe_1.0e+06            1.0   2.0   2.0   11.0  
mp_complex_pipe_1.0e+07            3.0   5.0   6.0   59.0  
mp_complex_queue_1.0e+06           1.0   1.0   1.0  150.0  
mp_complex_queue_1.0e+07           1.0   1.0   2.0  146.0  
mp_simple_pipe_1.0e+06             2.0   2.0   3.0   11.0  
mp_simple_pipe_1.0e+07             3.0   5.0   7.0   47.0  
mp_simple_queue_1.0e+06            2.0   2.0   3.0   12.0  
mp_simple_queue_1.0e+07            3.0   5.0   6.0   40.0  
sequential_1.0e+06                 2.0   2.0   3.0  161.0  
sequential_1.0e+07                 3.0   5.0   6.0  326.0  
sequential_1.0e+08                15.0  21.0  28.0   37.0

In [24]:
temp_ = a_list2[2].copy()

for col in temp_.columns:
    temp_[col] = temp_[col] / temp_[col].min()

temp_ = temp_.replace([np.inf, -np.inf], np.nan)    
temp_['sum'] = temp_.iloc[:,1:].sum(axis=1)
temp_

name                             milliseconds                                \
index                                   count       mean        std min 25%   
mp_complex_2_queues_to_1_1.0e+06     1.033262   1.128262   1.000000 NaN NaN   
mp_complex_2_queues_to_1_1.0e+07     1.043991   1.000000   1.469364 NaN NaN   
mp_complex_pipe_1.0e+06              1.032189   1.823392   1.882324 NaN NaN   
mp_complex_pipe_1.0e+07              1.068670   4.227839   5.689184 NaN NaN   
mp_complex_queue_1.0e+06             1.024678   1.085356   1.004982 NaN NaN   
mp_complex_queue_1.0e+07             1.028970   1.183698   4.591082 NaN NaN   
mp_simple_pipe_1.0e+06               1.007511   2.570373   1.090105 NaN NaN   
mp_simple_pipe_1.0e+07               1.069742   4.688520   4.036177 NaN NaN   
mp_simple_queue_1.0e+06              1.047210   2.564316   1.059413 NaN NaN   
mp_simple_queue_1.0e+07              1.032189   4.805719   3.870136 NaN NaN   
sequential_1.0e+06                   1.002146   2.812741  11.552679 NaN NaN   
sequential_1.0e+07                   1.000000   4.537013   3.769096 NaN NaN   
sequential_1.0e+08                   1.046137  20.172425  19.365741 NaN NaN   

name                                                              sum  
index                              50%   75%    99%   max              
mp_complex_2_queues_to_1_1.0e+06   1.0   1.0   1.38   1.0    6.508262  
mp_complex_2_queues_to_1_1.0e+07   1.0   1.0   2.00   5.5   11.969364  
mp_complex_pipe_1.0e+06            1.0   2.0   2.39   5.0   14.095716  
mp_complex_pipe_1.0e+07            3.0   5.0   6.00  27.0   50.917023  
mp_complex_queue_1.0e+06           1.0   1.0   1.00   1.0    6.090339  
mp_complex_queue_1.0e+07           1.0   1.0   2.00  28.0   37.774781  
mp_simple_pipe_1.0e+06             2.0   2.0   3.00   4.5   15.160479  
mp_simple_pipe_1.0e+07             3.0   5.0   7.00   5.5   29.224697  
mp_simple_queue_1.0e+06            2.0   2.0   3.00   4.0   14.623729  
mp_simple_queue_1.0e+07            3.0   5.0   6.00   3.5   26.175854  
sequential_1.0e+06                 2.0   2.0   3.00  71.0   92.365420  
sequential_1.0e+07                 3.0   5.0   6.00   7.0   29.306108  
sequential_1.0e+08                14.0  22.0  28.00  14.5  118.038166